## Importing libraries, modules

In [37]:
import os
import gzip
import numpy as np
import tensorflow as tf
import scipy.misc
from datetime import datetime

## Global parameters

In [38]:
gpu = 0
epochs = 100
batch_size = 100
image_shape = [28, 28, 1]

## Loading data utility

In [39]:
def read32(bytestream):
    dt = np.dtype(np.uint32).newbyteorder('>')
    return np.frombuffer(bytestream.read(4), dtype=dt)[0]

def load_train_data():
    mnist_dir = '/mnt/cube/f1fan/data/mnist'
    train_image_path = os.path.join(mnist_dir, 'train-images-idx3-ubyte.gz')
    train_label_path = os.path.join(mnist_dir, 'train-labels-idx1-ubyte.gz')

    with gzip.open(train_image_path) as image_stream, gzip.open(train_label_path) as label_stream:
        magic_image, magic_label = read32(image_stream), read32(label_stream)
        if magic_image != 2051 or magic_label != 2049:
            raise ValueError('Invalid magic number')

        image_count, label_count = read32(image_stream), read32(label_stream)
        row_count = read32(image_stream)
        col_count = read32(image_stream)

        label_buffer = label_stream.read(label_count)
        train_labels = np.frombuffer(label_buffer, dtype=np.uint8)

        image_buffer = image_stream.read(row_count * col_count * image_count)
        train_images = np.frombuffer(image_buffer, dtype=np.uint8)
        train_images = train_images.reshape(image_count, row_count, col_count, 1)

        return train_images, train_labels

def load_test_data():
    mnist_dir = '/mnt/cube/f1fan/data/mnist'
    test_image_path = os.path.join(mnist_dir, 't10k-images-idx3-ubyte.gz')
    test_label_path = os.path.join(mnist_dir, 't10k-labels-idx1-ubyte.gz')

    with gzip.open(test_image_path) as image_stream, gzip.open(test_label_path) as label_stream:
        magic_image, magic_label = read32(image_stream), read32(label_stream)
        if magic_image != 2051 or magic_label != 2049:
            raise ValueError('Invalid magic number')

        image_count, label_count = read32(image_stream), read32(label_stream)
        row_count = read32(image_stream)
        col_count = read32(image_stream)

        label_buffer = label_stream.read(label_count)
        test_labels = np.frombuffer(label_buffer, dtype=np.uint8)

        image_buffer = image_stream.read(row_count * col_count * image_count)
        test_images = np.frombuffer(image_buffer, dtype=np.uint8)
        test_images = test_images.reshape(image_count, row_count, col_count, 1)

        return test_images, test_labels
    
def normalize(images):
    '''
    Normalize the intensity values from [0, 255] into [-1, 1].
        images: Image array to normalize. Require each intensity value
                ranges from 0 to 255.
    Return normalized image array.
    '''
    return 1.0 * np.array(images) / 255 * 2.0 - 1.0

## Layer utility

In [40]:
def conv2d(scope, input_layer, output_dim, use_bias=True,
            filter_size=3, strides=[1, 1, 1, 1]):
    input_dim = input_layer.get_shape().as_list()[-1]

    with tf.variable_scope(scope):
        conv_filter = tf.get_variable(
            'conv_weight',
            shape = [filter_size, filter_size, input_dim, output_dim],
            dtype = tf.float32,
            initializer = tf.contrib.layers.variance_scaling_initializer(),
            regularizer = tf.contrib.layers.l2_regularizer(scale=0.0002)
        )
        conv = tf.nn.conv2d(input_layer, conv_filter, strides, 'SAME')

        if use_bias:
            bias = tf.get_variable(
                'conv_bias',
                shape = [output_dim],
                dtype = tf.float32,
                initializer = tf.constant_initializer(0.0)
            )

            output_layer = tf.nn.bias_add(conv, bias)
            output_layer = tf.reshape(output_layer, conv.get_shape())
        else:
            output_layer = conv

        return output_layer

def batch_norm(scope, input_layer, is_training, reuse):

    output_layer = tf.contrib.layers.batch_norm(
        input_layer,
        decay = 0.9,
        scale = True,
        epsilon = 1e-5,
        is_training = is_training,
        reuse = reuse,
        scope = scope
    )

    '''
    with tf.variable_scope(scope, reuse=reuse):
        input_dim = input_layer.get_shape().as_list()[-1]
        mean, variance = tf.nn.moments(input_layer, [0, 1, 2])
        beta = tf.get_variable(
            'bn_beta',
            shape = [input_dim],
            dtype = tf.float32,
            initializer = tf.constant_initializer(0.0)
        )
        gamma = tf.get_variable(
            'bn_gamma',
            shape = [input_dim],
            dtype = tf.float32,
            initializer = tf.constant_initializer(1.0)
        )

        output_layer = tf.nn.batch_normalization(input_layer, mean, variance,
                                                 beta, gamma, 0.00001)
    '''
    return output_layer

def lrelu(input_layer, leak=0.2):
    #output_layer = tf.nn.relu(input_layer)
    output_layer = tf.maximum(input_layer, leak * input_layer)
    #output_layer = input_layer * tf.sigmoid(input_layer)
    return output_layer

def fully_connected(scope, input_layer, output_dim):
    input_dim = input_layer.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope):
        fc_weight = tf.get_variable(
            'fc_weight',
            shape = [input_dim, output_dim],
            dtype = tf.float32,
            initializer = tf.contrib.layers.variance_scaling_initializer(),
            regularizer = tf.contrib.layers.l2_regularizer(scale=0.0002)            
        )

        fc_bias = tf.get_variable(
            'fc_bias',
            shape = [output_dim],
            dtype = tf.float32,
            initializer = tf.constant_initializer(0.0)
        )

        output_layer = tf.matmul(input_layer, fc_weight) + fc_bias

        return output_layer

def avg_pool(scope, input_layer, ksize=None, strides=[1, 2, 2, 1]):
    if ksize is None:
        ksize = strides

    with tf.variable_scope(scope):
        output_layer = tf.nn.avg_pool(input_layer, ksize, strides, 'VALID')
        return output_layer

## Building models

In [41]:
def build_model(input_image, is_training, reuse):
    with tf.variable_scope('models', reuse=reuse):
        filter_dim = 64
        batch_size = input_image.get_shape().as_list()[0]

        h0_conv = conv2d('h0_conv', input_image, filter_dim, filter_size=5, strides=[1, 2, 2, 1])
        h0 = lrelu(h0_conv)

        h1_conv = conv2d('h1_conv', h0, filter_dim * 2, filter_size=5, strides=[1, 2, 2, 1])
        h1_bn = batch_norm('h1_bn', h1_conv, is_training, reuse)
        h1 = lrelu(h1_bn)

        h2_conv = conv2d('h2_conv', h1, filter_dim * 4, filter_size=5, strides=[1, 2, 2, 1])
        h2_bn = batch_norm('h2_bn', h2_conv, is_training, reuse)
        h2 = lrelu(h2_bn)

        h3_conv = conv2d('h3_conv', h2, filter_dim * 8, filter_size=5, strides=[1, 2, 2, 1])
        h3_bn = batch_norm('h3_bn', h3_conv, is_training, reuse)
        h3 = lrelu(h3_bn)

        fc = fully_connected('fc', tf.reshape(h3, [batch_size, -1]), 10)
        return tf.nn.softmax(fc), fc

In [42]:
def build_train_op(batch_size, image_shape):
    [height, width, channels] = image_shape
    batch_shape = [batch_size, height, width, channels]
    train_image_placeholder = tf.placeholder(
        tf.float32,
        shape = batch_shape,
        name = 'train_images'
    )
    train_label_placeholder = tf.placeholder(
        tf.int32,
        shape = [batch_size, ],
        name = 'train_labels'
    )
    
    prob, logits = build_model(train_image_placeholder, True, False)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels = train_label_placeholder,
        logits = logits
    )
    
    train_step = tf.Variable(initial_value=0, trainable=False)
    prediction = tf.equal(tf.cast(tf.argmax(prob, axis=1), tf.int32), train_label_placeholder)
    train_loss = tf.reduce_mean(loss)
    train_accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    train_vars = [x for x in tf.trainable_variables() if 'models' in x.name]
    optimizer = tf.train.AdamOptimizer(0.01)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(train_loss, global_step=train_step, var_list=train_vars)
    
    return train_image_placeholder, train_label_placeholder, train_loss, train_accuracy, train_op

In [43]:
def build_test_op(batch_size, image_shape):
    [height, width, channels] = image_shape
    batch_shape = [batch_size, height, width, channels]
    test_image_placeholder = tf.placeholder(
        tf.float32,
        shape = batch_shape,
        name = 'test_images'
    )
    test_label_placeholder = tf.placeholder(
        tf.int32,
        shape = [batch_size, ],
        name = 'test_labels'
    )
    
    prob, logits = build_model(test_image_placeholder, False, True)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels = test_label_placeholder,
        logits = logits
    )
    prediction = tf.equal(tf.cast(tf.argmax(prob, axis=1), tf.int32), test_label_placeholder)
    test_loss = tf.reduce_mean(loss)
    test_accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    return test_image_placeholder, test_label_placeholder, test_loss, test_accuracy

In [44]:
def main(sess):
    train_images, train_labels = load_train_data()
    test_images, test_labels = load_test_data()
    
    train_image_placeholder, train_label_placeholder, train_loss, train_accuracy, train_op = build_train_op(batch_size, image_shape)
    test_image_placeholder, test_label_placeholder, test_loss, test_accuracy = build_test_op(batch_size, image_shape)

    all_initializer_op = tf.global_variables_initializer()
    sess.run(all_initializer_op)
    
    global_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='')
    print('Global variables:')
    for i,var in enumerate(global_variables):
        print('{}, {}, {}'.format(i, var.name, var.get_shape()))
    
    train_loss_list, train_accuracy_list = [], []
    test_loss_list, test_accuracy_list = [], []
    
    for ep in xrange(epochs):
        shuffle = np.random.permutation(train_images.shape[0])
        
        _loss = 0.0
        _accuracy = 0.0
        for bc in xrange(train_images.shape[0] // batch_size):
            batch_images = normalize(train_images[shuffle[bc * batch_size : (bc + 1) * batch_size]])
            batch_labels = train_labels[shuffle[bc * batch_size : (bc + 1) * batch_size]]
            
            sess.run(train_op, feed_dict={train_image_placeholder: batch_images, 
                                          train_label_placeholder: batch_labels})
            batch_loss, batch_accuracy = sess.run([train_loss, train_accuracy],
                                                  feed_dict={train_image_placeholder: batch_images,
                                                             train_label_placeholder: batch_labels})
            _loss += batch_loss
            _accuracy += batch_accuracy
        
        _loss /= (train_images.shape[0] // batch_size)
        _accuracy /= (train_images.shape[0] // batch_size)
        train_loss_list.append(_loss)
        train_accuracy_list.append(_accuracy)
        
        if ep % 5 == 0:
            print ('Epoch {}:'.format(ep))
            print ('    Train loss {}, Train accuracy {}'.format(_loss, _accuracy))

        _loss = 0.0
        _accuracy = 0.0
        for bc in xrange(test_images.shape[0] // batch_size):
            batch_images = normalize(test_images[bc * batch_size : (bc + 1) * batch_size])
            batch_labels = test_labels[bc * batch_size : (bc + 1) * batch_size]
            
            batch_loss, batch_accuracy = sess.run([test_loss, test_accuracy],
                                                  feed_dict={test_image_placeholder: batch_images,
                                                             test_label_placeholder: batch_labels})
            _loss += batch_loss
            _accuracy += batch_accuracy
        
        _loss /= (test_images.shape[0] // batch_size)
        _accuracy /= (test_images.shape[0] // batch_size)
        test_loss_list.append(_loss)
        test_accuracy_list.append(_accuracy)
        if ep % 5 == 0:
            print ('    Test loss {}, Test accuracy {}'.format(_loss, _accuracy))

In [45]:
if __name__ == '__main__':
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True

    graph = tf.Graph()
    with graph.as_default():
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
        with tf.device('/gpu:0'):
            with tf.Session(config=config) as sess:
                with tf.variable_scope('SimpleCNN', reuse=None):
                    main(sess)

Global variables:
0, SimpleCNN/models/h0_conv/conv_weight:0, (5, 5, 1, 64)
1, SimpleCNN/models/h0_conv/conv_bias:0, (64,)
2, SimpleCNN/models/h1_conv/conv_weight:0, (5, 5, 64, 128)
3, SimpleCNN/models/h1_conv/conv_bias:0, (128,)
4, SimpleCNN/models/h1_bn/beta:0, (128,)
5, SimpleCNN/models/h1_bn/gamma:0, (128,)
6, SimpleCNN/models/h1_bn/moving_mean:0, (128,)
7, SimpleCNN/models/h1_bn/moving_variance:0, (128,)
8, SimpleCNN/models/h2_conv/conv_weight:0, (5, 5, 128, 256)
9, SimpleCNN/models/h2_conv/conv_bias:0, (256,)
10, SimpleCNN/models/h2_bn/beta:0, (256,)
11, SimpleCNN/models/h2_bn/gamma:0, (256,)
12, SimpleCNN/models/h2_bn/moving_mean:0, (256,)
13, SimpleCNN/models/h2_bn/moving_variance:0, (256,)
14, SimpleCNN/models/h3_conv/conv_weight:0, (5, 5, 256, 512)
15, SimpleCNN/models/h3_conv/conv_bias:0, (512,)
16, SimpleCNN/models/h3_bn/beta:0, (512,)
17, SimpleCNN/models/h3_bn/gamma:0, (512,)
18, SimpleCNN/models/h3_bn/moving_mean:0, (512,)
19, SimpleCNN/models/h3_bn/moving_variance:0, (51